In [ ]:
!pip install transformers

In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import EncoderDecoderModel, EncoderDecoderConfig
from transformers import BertModel, BertLMHeadModel, BertConfig

In [ ]:
RANDOM_SEED=42
np.random.seed(RANDOM_SEED)

MAX_LENGTH = 41

In [ ]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train, df_val = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")

In [ ]:
train_data_tokens = tokenizer(df_train['data'].tolist(), padding='max_length', truncation=True, max_length=MAX_LENGTH)
val_data_tokens = tokenizer(df_val['data'].tolist(), padding='max_length', truncation=True, max_length=MAX_LENGTH)
test_data_tokens = tokenizer(df_test['data'].tolist(), padding='max_length', truncation=True, max_length=MAX_LENGTH)

train_target_tokens = tokenizer(df_train['label'].tolist(), padding='max_length', truncation=True, max_length=MAX_LENGTH)
val_target_tokens = tokenizer(df_val['label'].tolist(), padding='max_length', truncation=True, max_length=MAX_LENGTH)

In [ ]:
class LangDataset(torch.utils.data.Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __getitem__(self, idx):
        item = {'input_ids' : torch.tensor(self.data[idx].ids),
                'attention_mask': torch.tensor(self.data[idx].attention_mask),
                'labels': torch.tensor(self.targets[idx].ids)
        }
        return item

    def __len__(self):
        return len(self.data['input_ids'])

In [ ]:
train_dataset = LangDataset(train_data_tokens, train_target_tokens)
val_dataset = LangDataset(val_data_tokens, val_target_tokens)
test_dataset = LangDataset(test_data_tokens, test_data_tokens)

train_dataloader = DataLoader(train_dataset, batch_size=64)
val_dataloader = DataLoader(val_dataset, batch_size=16)

In [ ]:
encoder_config = BertConfig(vocab_size = len(tokenizer),
                    max_position_embeddings = MAX_LENGTH + 64, 
                    num_attention_heads = 6,
                    num_hidden_layers = 6)

encoder = BertModel(config=encoder_config)


decoder_config = BertConfig(vocab_size = len(tokenizer),
                    max_position_embeddings = MAX_LENGTH + 64, 
                    num_attention_heads = 6,
                    num_hidden_layers = 6,
                    is_decoder=True)  
decoder_config.is_decoder = True
decoder_config.add_cross_attention = True

decoder = BertLMHeadModel(config=decoder_config)


model = EncoderDecoderModel(encoder=encoder, decoder=decoder)

model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    num_train_epochs=30,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=10,
    save_strategy='steps',
    save_steps=400,
    eval_accumulation_steps=16
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 985
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 930
/usr/local/lib/python3.7/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:490: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss,Validation Loss
10,11.807700,11.744199
20,11.689200,11.535938
30,11.480600,11.311883
40,11.281700,11.127597
50,11.107500,10.951204
60,10.942700,10.761515
70,10.767100,10.560139
80,10.570300,10.354213
90,10.370500,10.155472
100,10.168100,9.956820


***** Running Evaluation *****
  Num examples = 110
  Batch size = 64
***** Running Evaluation *****
  Num examples = 110
  Batch size = 64
***** Running Evaluation *****
  Num examples = 110
  Batch size = 64
***** Running Evaluation *****
  Num examples = 110
  Batch size = 64
***** Running Evaluation *****
  Num examples = 110
  Batch size = 64
***** Running Evaluation *****
  Num examples = 110
  Batch size = 64
***** Running Evaluation *****
  Num examples = 110
  Batch size = 64
***** Running Evaluation *****
  Num examples = 110
  Batch size = 64
***** Running Evaluation *****
  Num examples = 110
  Batch size = 64
***** Running Evaluation *****
  Num examples = 110
  Batch size = 64
***** Running Evaluation *****
  Num examples = 110
  Batch size = 64
***** Running Evaluation *****
  Num examples = 110
  Batch size = 64
***** Running Evaluation *****
  Num examples = 110
  Batch size = 64
***** Running Evaluation *****
  Num examples = 110
  Batch size = 64
***** Running Evalua

TrainOutput(global_step=930, training_loss=3.0981063771832695, metrics={'train_runtime': 1356.6324, 'train_samples_per_second': 21.782, 'train_steps_per_second': 0.686, 'total_flos': 730878880031100.0, 'train_loss': 3.0981063771832695, 'epoch': 30.0})

In [ ]:
batch_size=16

id = []
data = []
label = []

for batch_idx in range(len(test_data_tokens.input_ids) // batch_size):
    id.extend(df_test.iloc[batch_idx*batch_size : (batch_idx + 1)*batch_size]['id'])
    data.extend(df_test.iloc[batch_idx*batch_size : (batch_idx + 1)*batch_size]['data'])
    test_data = torch.tensor(test_data_tokens.input_ids[batch_idx*batch_size : (batch_idx + 1)*batch_size])
    label.extend(trainer.model_wrapped.generate(test_data.to('cuda'),
                                decoder_start_token_id=tokenizer.cls_token_id))


In [ ]:
labels = [tokenizer.decode(lab, skip_special_tokens=True) for lab in label]

In [ ]:
df_res = pd.DataFrame({'id': id, 'data': data, 'label': labels})

In [ ]:
df_res['label'] = df_res['label'].str.replace(' ', '')

In [ ]:
df_res[['id', 'label']].to_csv('submission.csv', index=False)

In [ ]:
df_res

,id,data,label
0,0,05/10/2007,05-10-2007
1,1,elfter september 2007,11-09-2007
2,2,09 sa'wol 2077,09-04-2077
3,3,le vingt-huit mai 2077,28-05-2077
4,4,chile gu'wol 2007,07-09-2007
...,...,...,...
4651,4651,осмог јуна 2077,08-06-2077
4652,4652,sipil sibirwol 2077,11-11-2077
4653,4653,yuke i'wol 2049,06-02-2049
4654,4654,четырнадцатого 07 2049,14-07-2049
